In [1]:
import pandas as pd

In [35]:
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

In [3]:
df = pd.read_pickle('../../data/sample_data/annotated_df_iaa_sample_parsed.pkl')
df

,sen_tok,char,token,label,relation,annotator,institution,year,MDN,NotitieID,batch,legacy_rawfile,sen_id,tok,STM,STM_lvl,INS,INS_lvl,FAC,FAC_lvl,BER,BER_lvl,disregard,target,background,other
0,1-1,427-429,Op,type\_Background[1],_,avelli,vumc,2017,,175982538,pilot,notities_2017_deel1_cleaned.csv,175982538_1,1,False,NaN,False,NaN,False,NaN,False,NaN,False,False,True,[]
1,1-2,430-439,Dd-mm-jjjj,type\_Background[1],_,avelli,vumc,2017,,175982538,pilot,notities_2017_deel1_cleaned.csv,175982538_1,2,False,NaN,False,NaN,False,NaN,False,NaN,False,False,True,[]
2,1-3,440-444,werd,type\_Background[1],_,avelli,vumc,2017,,175982538,pilot,notities_2017_deel1_cleaned.csv,175982538_1,3,False,NaN,False,NaN,False,NaN,False,NaN,False,False,True,[]
3,1-4,445-458,bovengenoemde,type\_Background[1],_,avelli,vumc,2017,,175982538,pilot,notities_2017_deel1_cleaned.csv,175982538_1,4,False,NaN,False,NaN,False,NaN,False,NaN,False,False,True,[]
4,1-5,459-466,patiënt,type\_Background[1],_,avelli,vumc,2017,,175982538,pilot,notities_2017_deel1_cleaned.csv,175982538_1,5,False,NaN,False,NaN,False,NaN,False,NaN,False,False,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2-18,563-567,hier,NaN,NaN,meskers,vumc,2017,,147501746,pilot,notities_2017_deel1_cleaned.csv,147501746_2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,2-19,568-572,veel,NaN,NaN,meskers,vumc,2017,,147501746,pilot,notities_2017_deel1_cleaned.csv,147501746_2,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
377,2-20,573-579,zorgen,NaN,NaN,meskers,vumc,2017,,147501746,pilot,notities_2017_deel1_cleaned.csv,147501746_2,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,2-21,580-582,om,NaN,NaN,meskers,vumc,2017,,147501746,pilot,notities_2017_deel1_cleaned.csv,147501746_2,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
levels = [col for col in df.columns if '_lvl' in col]
domains = [col[:3] for col in levels]

In [12]:
overview = df.groupby(['sen_id', 'annotator'])[domains].any().stack().unstack(1)
overview.loc[overview.any(axis=1)]

,annotator,avelli,bos,meskers
sen_id,,,,
147501746_2,STM,True,True,False
175982538_2,FAC,True,False,False
175982538_3,FAC,True,True,True
223867383_2,FAC,True,False,True
223867383_3,FAC,True,False,False


In [33]:
get_levels = lambda g, level: g[level].apply(lambda s: [i for i in s.unique() if i==i])
grouped = df.groupby(['sen_id', 'annotator'])   

overview = pd.concat([get_levels(grouped, level) for level in levels], axis=1).stack().unstack(1)
overview.loc[overview.any(axis=1)]

,annotator,avelli,bos,meskers
sen_id,,,,
147501746_2,STM_lvl,"[1.0, 0.0]","[1.0, 0.0]",[]
175982538_3,FAC_lvl,[1.0],[2.0],[4.0]
223867383_3,FAC_lvl,[],[],[3.0]


In [4]:
cols = ['label', 'relation']
idx = ['sen_id', 'tok', 'token']
annotators = df.annotator.unique()

def assign_annotator(df, annotator):
    to_rename = {'label': f'label_{annotator}', 'relation': f'relation_{annotator}'}
    return df.set_index(idx).query(f"annotator == '{annotator}'")[cols].rename(columns=to_rename)

dfs = [assign_annotator(df, ann) for ann in annotators]
overview = pd.concat(dfs, axis=1).reset_index()
non_empty_rows = overview.iloc[:, 3:].applymap(lambda x: x == x and x != '_').any(axis=1)
non_empty_sen_ids = overview.loc[non_empty_rows].sen_id.unique()
overview.loc[overview.sen_id.isin(non_empty_sen_ids)]

,sen_id,tok,token,label_avelli,relation_avelli,label_bos,relation_bos,label_meskers,relation_meskers
14,147501746_2,1,Mw,_,_,_,_,NaN,NaN
15,147501746_2,2,was,_,_,_,_,NaN,NaN
16,147501746_2,3,emotioneel,.B152: Stemming[1]|STM 1[2],14-3[2_1],.B152: Stemming[1]|STM 1[2],14-3[2_1],NaN,NaN
17,147501746_2,4,",",_,_,_,_,NaN,NaN
18,147501746_2,5,schoonzoon,stm\_reaction[3],_,_,_,NaN,NaN
...,...,...,...,...,...,...,...,...,...
122,223867383_3,5,en,type\_Background[2],_,type\_Background[1],_,type\_Background[2]|type\_Background[3],_
123,223867383_3,6,meerdere,type\_Background[2],_,type\_Background[1],_,type\_Background[2]|type\_Background[3],_
124,223867383_3,7,malen,type\_Background[2],_,type\_Background[1],_,type\_Background[2]|type\_Background[3],_
125,223867383_3,8,gevallen,type\_Background[2],_,type\_Background[1],_,type\_Background[2]|type\_Background[3],_
